In [68]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
stopWords = set(stopwords.words('english'))

In [69]:
df1 = pd.read_csv('Project 4 files/Resources/cpj_salary_location_cleaned.csv')

In [70]:
df1=df1[["Job Title","Location","Mean_Salary","Description","Category","Skills","Tools","state"]]
df1=df1.dropna()
df1=df1.drop_duplicates()
df1=df1[df1["Mean_Salary"]!=0]
df1

,Job Title,Location,Mean_Salary,Description,Category,Skills,Tools,state
2,Junior Data Analyst,"Louisville, KY 40223",35750,Basic Function: Develop and process targeted m...,Data Analyst,Data Analysis,"SQL, R, Excel",KY
4,Marketing Data Analyst,"Sarasota, FL 34211",42500,Mercedes Scientific is a national leader in th...,Data Analyst,,R,FL
7,Power BI/Data Analyst,"Dallas-Fort Worth, TX•Temporarily Remote",95000,About our TeamThe Ethos Group IT team consists...,Data Analyst,Statistics,"SQL, R, Excel, Power BI, Tableau",TX
9,Online Data Analyst - USA,"Annapolis, MD•Remote",14500,Company Info: We help companies test and impro...,Data Analyst,,"R, AI, Machine Learning",MD
10,Online Data Analyst - USA,"New Orleans, LA•Remote",14500,Company Info: We help companies test and impro...,Data Analyst,,"R, AI, Machine Learning",LA
...,...,...,...,...,...,...,...,...
15646,Full Stack Developer (REMOTE),"Tampa, FL 33601•Remote",47500,Join us as we pursue our mission to remove the...,Business Intelligence Developer,,"Python, Spark, JavaScript, R, Cloud, HTML, CSS...",FL
15668,Data Analyst II Database developer II Entry le...,"Poway, CA•Temporarily Remote",65000,SQL Developer (Training)Responsibilities· Help...,Business Intelligence Developer,ETL,"SQL, R, Power BI, Tableau",CA
15671,Workday Business Intelligence Developer,"Las Vegas, NV 89119 (Paradise area)",80000,Workday Business Intelligence Developer\nThank...,Business Intelligence Developer,,"R, Cloud, Excel, Java",NV
15689,Data Developer,"Research Triangle Park, NC 27709",52011,TITLE: IT Data Analytics/Data Developer 1\nDut...,Business Intelligence Developer,,"SQL, R, Power BI, Tableau, Scala",NC


In [71]:
# Python
df1['Python'] = df1['Tools'].apply(lambda x: 1 if 'python' in x.lower() else 0)

# R studio 
df1['R'] = df1['Tools'].apply(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)

# SQL 
df1['SQL'] = df1['Tools'].apply(lambda x: 1 if 'sql' in x.lower() else 0)

# AWS 
df1['AWS'] = df1['Tools'].apply(lambda x: 1 if 'aws' in x.lower() else 0)

# Excel
df1['Excel'] = df1['Tools'].apply(lambda x: 1 if 'excel' in x.lower() else 0)

# Google Cloud
df1['GCP'] = df1['Tools'].apply(lambda x: 1 if 'google cloud' in x.lower() or 'gcp' in x.lower() else 0)

# Microsoft Azure
df1['Azure'] = df1['Tools'].apply(lambda x: 1 if 'microsoft azure' in x.lower() or 'azure' in x.lower() else 0)

# Spark
df1['Spark'] = df1['Tools'].apply(lambda x: 1 if 'spark' in x.lower() else 0)

# Tableau
df1['Tableau'] = df1['Tools'].apply(lambda x: 1 if 'tableau' in x.lower() else 0)

# Keras
df1['Keras'] = df1['Tools'].apply(lambda x: 1 if 'keras' in x.lower() else 0)

#NoSQL
df1['NoSQL'] = df1['Tools'].apply(lambda x: 1 if 'nosql' in x.lower() else 0)

#Machine Learning
df1['Machine_Learning'] = df1['Tools'].apply(lambda x: 1 if 'machine learning' in x.lower() else 0)

#Hadoop
df1['Hadoop'] = df1['Tools'].apply(lambda x: 1 if 'hadoop' in x.lower() else 0)

df1['AI'] = df1['Tools'].apply(lambda x: 1 if 'ai' in x.lower() else 0)

df1['Power_BI'] = df1['Tools'].apply(lambda x: 1 if 'power bi' in x.lower() else 0)


In [72]:
df1=df1.drop(["Skills","Tools"],axis=1)

In [73]:
# Classifying job levels into each category
def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'vp' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower()or 'experienced' in title.lower() or 'iii' in title.lower() or 'research' in title.lower() or 'manager' in title.lower():
        return 'Senior'
    elif 'intermediate' in title.lower() or 'staff' in title.lower() or 'ii' in title.lower():
        return 'Mid'
    elif 'jr' in title.lower() or 'junior' in title.lower() or 'intern' in title.lower() or 'student' in title.lower()or 'associate' in title.lower():
        return 'Junior'
    else:
        return 'Not Specified'

In [100]:
df1['Seniority'] = df1['Job Title'].apply(seniority)
# Categorize salary
def salary_category(salary):
    if salary<5000:
        return 0
    if 50000 <= salary < 75000:
        return 1
    if 75000 <= salary < 100000:
        return 2
    if 100000 <= salary < 125000:
        return 3
    if 125000 <= salary < 150000:
        return 4
    if 150000 <= salary < 175000:
        return 5
    if 175000 <= salary < 200000:
        return 6
    else:
        return 7
target = df1['Mean_Salary'].apply(salary_category)
len(target)

1781

In [76]:
df1=df1.drop("Location",axis=1)


In [77]:
df1

,Job Title,Mean_Salary,Description,Category,state,Python,R,SQL,AWS,Excel,...,Azure,Spark,Tableau,Keras,NoSQL,Machine_Learning,Hadoop,AI,Power_BI,Seniority
2,Junior Data Analyst,35750,Basic Function: Develop and process targeted m...,Data Analyst,KY,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,Junior
4,Marketing Data Analyst,42500,Mercedes Scientific is a national leader in th...,Data Analyst,FL,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Not Specified
7,Power BI/Data Analyst,95000,About our TeamThe Ethos Group IT team consists...,Data Analyst,TX,0,0,1,0,1,...,0,0,1,0,0,0,0,0,1,Not Specified
9,Online Data Analyst - USA,14500,Company Info: We help companies test and impro...,Data Analyst,MD,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,Not Specified
10,Online Data Analyst - USA,14500,Company Info: We help companies test and impro...,Data Analyst,LA,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,Not Specified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15646,Full Stack Developer (REMOTE),47500,Join us as we pursue our mission to remove the...,Business Intelligence Developer,FL,1,0,0,0,1,...,0,1,0,0,0,0,1,0,0,Not Specified
15668,Data Analyst II Database developer II Entry le...,65000,SQL Developer (Training)Responsibilities· Help...,Business Intelligence Developer,CA,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,Mid
15671,Workday Business Intelligence Developer,80000,Workday Business Intelligence Developer\nThank...,Business Intelligence Developer,NV,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Not Specified
15689,Data Developer,52011,TITLE: IT Data Analytics/Data Developer 1\nDut...,Business Intelligence Developer,NC,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,Not Specified


In [78]:
df1['Description'] = df1['Description'].apply(lambda x: ' '.join(x.split()))
df1['Description'] = df1['Description'].apply(lambda x: re.sub(r'[^\w\s]','', x.lower()))
df1['Description'] = df1['Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopWords]))   

In [79]:
df1

,Job Title,Mean_Salary,Description,Category,state,Python,R,SQL,AWS,Excel,...,Azure,Spark,Tableau,Keras,NoSQL,Machine_Learning,Hadoop,AI,Power_BI,Seniority
2,Junior Data Analyst,35750,basic function develop process targeted mailin...,Data Analyst,KY,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,Junior
4,Marketing Data Analyst,42500,mercedes scientific national leader distributi...,Data Analyst,FL,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Not Specified
7,Power BI/Data Analyst,95000,teamthe ethos group team consists members work...,Data Analyst,TX,0,0,1,0,1,...,0,0,1,0,0,0,0,0,1,Not Specified
9,Online Data Analyst - USA,14500,company info help companies test improve machi...,Data Analyst,MD,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,Not Specified
10,Online Data Analyst - USA,14500,company info help companies test improve machi...,Data Analyst,LA,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,Not Specified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15646,Full Stack Developer (REMOTE),47500,join us pursue mission remove barriers data ac...,Business Intelligence Developer,FL,1,0,0,0,1,...,0,1,0,0,0,0,1,0,0,Not Specified
15668,Data Analyst II Database developer II Entry le...,65000,sql developer trainingresponsibilities help wr...,Business Intelligence Developer,CA,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,Mid
15671,Workday Business Intelligence Developer,80000,workday business intelligence developer thank ...,Business Intelligence Developer,NV,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Not Specified
15689,Data Developer,52011,title data analyticsdata developer 1 duties as...,Business Intelligence Developer,NC,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,Not Specified


In [80]:
df=pd.read_csv('static/resources/data.csv')

In [81]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable
1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD)
2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD)
3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD)
4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable


In [82]:
df.dtypes

Job Title             object
Salary Estimate       object
Job Description       object
Rating               float64
Company Name          object
Location              object
Size                  object
Founded               object
Type of ownership     object
Industry              object
Sector                object
Revenue               object
dtype: object

In [83]:
#Get the datas with salary with glassdoor estimate
df['Salary_est'] = df['Salary Estimate'].apply(lambda x: 1 if 'glassdoor est.' in x.lower() else 0)
df = df[df['Salary_est'] == 1]

In [84]:
# Calculate average salary and drop non-useful columns
salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0])
minus_Kd = salary.apply(lambda x: x.replace('K','').replace('$',''))


df['min_salary'] = minus_Kd.apply(lambda x: int(x.split('-')[0]))
df['max_salary'] = minus_Kd.apply(lambda x: int(x.split('-')[1]))
df['Salary'] = (df.min_salary+df.max_salary)*1000/2
df = df.drop(['Salary Estimate', 'Salary_est', 'min_salary', 'max_salary'],1)

In [85]:
# Company name text only
df['Company Name'] = df.apply(lambda x: x['Company Name'] if x['Rating'] <0 else x['Company Name'].split('\n')[0], axis = 1)

In [86]:
# Create new features such as city & state 
df['City'] = df['Location'].apply(lambda x: x.split(',')[0] if ',' in x.lower() else x)
df['State'] = df['Location'].apply(lambda x: x.split(',')[1] if ',' in x.lower() else x) 
df = df.drop(['Location'],1)
df.head()

,Job Title,Job Description,Rating,Company Name,Size,Founded,Type of ownership,Industry,Sector,Revenue,Salary,City,State
0,Sr Data Scientist,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,154000.0,New York,NY
1,Data Scientist,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),154000.0,Chantilly,VA
2,Data Scientist,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),154000.0,Boston,MA
3,Data Scientist,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),154000.0,Newton,MA
4,Data Scientist,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,154000.0,New York,NY


In [87]:
# Calculate the age of company 
df['Founded'] = df['Founded'].apply(lambda x: np.nan if x in ['Company - Private', '-1', 'Company - Public', 'Unknown', 'Contract', 'Nonprofit Organization', 'Self-employed'] else x)
df['Age'] = df['Founded'].apply(lambda x: x if x is np.nan else 2021 - int(x))
df = df.drop(['Founded'],1)

In [88]:
# Python
df['Python'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)

# R studio 
df['R'] = df['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)

# SQL 
df['SQL'] = df['Job Description'].apply(lambda x: 1 if 'sql' in x.lower() else 0)

# AWS 
df['AWS'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)

# Excel
df['Excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)

# Google Cloud
df['GCP'] = df['Job Description'].apply(lambda x: 1 if 'google cloud' in x.lower() or 'gcp' in x.lower() else 0)

# Microsoft Azure
df['Azure'] = df['Job Description'].apply(lambda x: 1 if 'microsoft azure' in x.lower() or 'azure' in x.lower() else 0)

# Spark
df['Spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)

# PyTorch
df['PyTorch'] = df['Job Description'].apply(lambda x: 1 if 'pytorch' in x.lower() else 0)

# TensorFlow
df['TensorFlow'] = df['Job Description'].apply(lambda x: 1 if 'tensorflow' in x.lower() else 0)

# Tableau
df['Tableau'] = df['Job Description'].apply(lambda x: 1 if 'tableau' in x.lower() else 0)

# Keras
df['Keras'] = df['Job Description'].apply(lambda x: 1 if 'keras' in x.lower() else 0)

#NoSQL
df['NoSQL'] = df['Job Description'].apply(lambda x: 1 if 'nosql' in x.lower() else 0)

#Sci-kit Learn
df['Scikit-Learn'] = df['Job Description'].apply(lambda x: 1 if 'scikit learn' in x.lower() else 0)

#Machine Learning
df['Machine_Learning'] = df['Job Description'].apply(lambda x: 1 if 'machine learning' in x.lower() else 0)

#Hadoop
df['Hadoop'] = df['Job Description'].apply(lambda x: 1 if 'hadoop' in x.lower() else 0)

# Scala
df['Scala'] = df['Job Description'].apply(lambda x: 1 if 'scala' in x.lower() else 0)

# Data Brick
df['Data_Brick'] = df['Job Description'].apply(lambda x: 1 if 'data brick' in x.lower() else 0)

In [89]:
# Convert size to lower case
df['Size'] = df['Size'].str.lower()
df.head()

,Job Title,Job Description,Rating,Company Name,Size,Type of ownership,Industry,Sector,Revenue,Salary,...,PyTorch,TensorFlow,Tableau,Keras,NoSQL,Scikit-Learn,Machine_Learning,Hadoop,Scala,Data_Brick
0,Sr Data Scientist,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,1001 to 5000 employees,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,154000.0,...,0,0,0,0,0,0,1,0,0,0
1,Data Scientist,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,5001 to 10000 employees,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),154000.0,...,0,0,0,0,0,0,1,1,0,0
2,Data Scientist,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,1001 to 5000 employees,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),154000.0,...,0,0,0,0,0,0,1,0,0,0
3,Data Scientist,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,501 to 1000 employees,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),154000.0,...,1,1,0,0,1,0,1,0,0,0
4,Data Scientist,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,51 to 200 employees,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,154000.0,...,0,0,0,0,0,0,1,0,0,0


In [90]:
df['Job Title'] = df['Job Title'].apply(lambda x: ' '.join(x.split()))
df['Job Title'] = df['Job Title'].apply(lambda x: re.sub(r'[^\w\s]',' ', x.lower()))
df['Job Description'] = df['Job Description'].apply(lambda x: ' '.join(x.split()))
df['Job Description'] = df['Job Description'].apply(lambda x: re.sub(r'[^\w\s]','', x.lower()))
df['Job Description'] = df['Job Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopWords]))   

In [91]:
# Classifying job titles into each category
def title_simplifier(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'analyst' in title.lower():
        return 'data analyst'
    elif 'machine learning' in title.lower():
        return 'machine learning engineer'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    else:
        return 'data science related jobs'


In [92]:
# Classifying job levels into each category
def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'vp' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower()or 'experienced' in title.lower() or 'iii' in title.lower() or 'research' in title.lower() or 'manager' in title.lower():
        return 'Senior'
    elif 'intermediate' in title.lower() or 'staff' in title.lower() or 'ii' in title.lower():
        return 'Mid'
    elif 'jr' in title.lower() or 'junior' in title.lower() or 'intern' in title.lower() or 'student' in title.lower()or 'associate' in title.lower():
        return 'Junior'
    else:
        return 'Not Specified'

In [93]:
# Process data based on above defined functions 
df['Job'] = df['Job Title'].apply(title_simplifier)
df['Seniority'] = df['Job Title'].apply(seniority)

In [94]:
# Categorize salary
def salary_category(salary):

    if 50000 <= salary < 75000:
        return 1
    if 75000 <= salary < 100000:
        return 2
    if 100000 <= salary < 125000:
        return 3
    if 125000 <= salary < 150000:
        return 4
    if 150000 <= salary < 175000:
        return 5
    if 175000 <= salary < 200000:
        return 6
    else:
        return 7
target = df['Salary'].apply(salary_category)

In [95]:
# Fill all NaN values in numeric & text features 
df['Age'] = df['Age'].fillna(0)
df = df.fillna('Unknown')

In [96]:
class TextSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

In [97]:
df1.columns

Index(['Job Title', 'Mean_Salary', 'Description', 'Category', 'state',
       'Python', 'R', 'SQL', 'AWS', 'Excel', 'GCP', 'Azure', 'Spark',
       'Tableau', 'Keras', 'NoSQL', 'Machine_Learning', 'Hadoop', 'AI',
       'Power_BI', 'Seniority'],
      dtype='object')

In [102]:
features= df1.drop("Mean_Salary",axis=1)
numeric_features= [c for c in df1.columns.values if c not in ['Job Title','state','Category','Seniority',"Description"]]
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
X_train.head(2)

,Job Title,Description,Category,state,Python,R,SQL,AWS,Excel,GCP,Azure,Spark,Tableau,Keras,NoSQL,Machine_Learning,Hadoop,AI,Power_BI,Seniority
924,Data Analyst,job summary functional analyst work within bus...,Data Analyst,TX,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,Not Specified
4651,Senior Machine Learning Visualization Engineer,machine learning engineer market intelligence ...,Machine Learning Engineer,VA,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,Senior


In [103]:
# Define text feature pipelines
Job_Title = Pipeline([
                ('selector', TextSelector(key='Job Title')),
                ('tfidf', TfidfVectorizer(stop_words='english'))
            ])


State = Pipeline([
                ('selector', TextSelector(key='state')),
                ('tfidf', TfidfVectorizer(stop_words='english'))
            ])

Category = Pipeline([
                ('selector', TextSelector(key='Category')),
                ('tfidf', TfidfVectorizer(stop_words='english'))
            ])

Seniority = Pipeline([
                ('selector', TextSelector(key='Seniority')),
                ('tfidf', TfidfVectorizer(stop_words='english'))
            ])
Description = Pipeline([
                ('selector', TextSelector(key='Description')),
                ('tfidf', TfidfVectorizer(stop_words='english'))
 ])
Job_Title.fit_transform(X_train)

<1424x617 sparse matrix of type '<class 'numpy.float64'>'
	with 4798 stored elements in Compressed Sparse Row format>

In [104]:
# Define numeric feature pipelines 

Python =  Pipeline([
                ('selector', NumberSelector(key='Python')),
                ('standard', StandardScaler())
            ])
R =  Pipeline([
                ('selector', NumberSelector(key='R')),
                ('standard', StandardScaler()),
            ])
SQL =  Pipeline([
                ('selector', NumberSelector(key='SQL')),
                ('standard', StandardScaler()),
            ])
AWS =  Pipeline([
                ('selector', NumberSelector(key='AWS')),
                ('standard', StandardScaler()),
            ])
Excel =  Pipeline([
                ('selector', NumberSelector(key='Excel')),
                ('standard', StandardScaler()),
            ])
GCP =  Pipeline([
                ('selector', NumberSelector(key='GCP')),
                ('standard', StandardScaler()),
            ])
Azure =  Pipeline([
                ('selector', NumberSelector(key='Azure')),
                ('standard', StandardScaler()),
            ])
Spark =  Pipeline([
                ('selector', NumberSelector(key='Spark')),
                ('standard', StandardScaler()),
            ])

Tableau =  Pipeline([
                ('selector', NumberSelector(key='Tableau')),
                ('standard', StandardScaler()),
            ])
Keras =  Pipeline([
                ('selector', NumberSelector(key='Keras')),
                ('standard', StandardScaler()),
            ])
NoSQL =  Pipeline([
                ('selector', NumberSelector(key='NoSQL')),
                ('standard', StandardScaler()),
            ])

Machine_Learning =  Pipeline([
                ('selector', NumberSelector(key='Machine_Learning')),
                ('standard', StandardScaler()),
            ])
Hadoop =  Pipeline([
                ('selector', NumberSelector(key='Hadoop')),
                ('standard', StandardScaler()),
            ])
AI =  Pipeline([
                ('selector', NumberSelector(key='AI')),
                ('standard', StandardScaler()),
            ])
Power_BI =  Pipeline([
                ('selector', NumberSelector(key='Power_BI')),
                ('standard', StandardScaler()),
            ])

In [106]:
# FeatureUnion for all feature vectors 
feats = FeatureUnion([('Job Title', Job_Title), 
                    ('Description', Description),
                      ('State', State),
                      ('Category', Category),
                      ('Seniority',Seniority),
                      ('Python', Python),
                      ('R', R),
                      ('SQL', SQL),
                      ('AWS', AWS),
                      ('Excel', Excel),
                      ('GCP', GCP),
                      ('Azure', Azure),
                      ('Spark', Spark),
                      ('Tableau', Tableau),
                      ('NoSQL', NoSQL),
                      ('Machine_Learning',Machine_Learning),
                      ('Hadoop', Hadoop),
                      ('AI',AI),
                      ('Power_BI',Power_BI)
                     ])

feature_processing = Pipeline([('feats', feats)])
feature_processing.fit_transform(X_train)

<1424x28188 sparse matrix of type '<class 'numpy.float64'>'
	with 394086 stored elements in Compressed Sparse Row format>

In [107]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

preds = pipeline.predict(X_test)
preds

array([7, 2, 3, 7, 3, 4, 1, 1, 7, 2, 2, 1, 2, 0, 7, 1, 2, 1, 2, 1, 3, 7,
       1, 1, 4, 2, 2, 2, 1, 4, 1, 2, 2, 1, 1, 2, 7, 3, 7, 3, 1, 7, 1, 2,
       1, 1, 2, 1, 3, 2, 3, 3, 2, 2, 7, 1, 1, 1, 2, 2, 2, 2, 2, 7, 2, 2,
       2, 3, 1, 2, 5, 7, 2, 7, 1, 4, 2, 2, 1, 7, 2, 2, 1, 1, 1, 7, 1, 2,
       2, 7, 2, 2, 2, 1, 7, 3, 4, 1, 3, 3, 2, 1, 2, 2, 4, 2, 1, 1, 1, 3,
       7, 1, 1, 3, 1, 1, 1, 4, 2, 4, 1, 4, 7, 1, 1, 2, 7, 2, 1, 6, 7, 1,
       1, 4, 4, 2, 1, 2, 1, 7, 1, 1, 1, 7, 7, 1, 4, 7, 2, 2, 2, 7, 1, 1,
       1, 2, 7, 2, 2, 2, 2, 2, 5, 7, 4, 2, 3, 1, 1, 1, 3, 2, 4, 2, 1, 7,
       4, 1, 1, 2, 1, 2, 2, 3, 2, 1, 7, 2, 2, 1, 2, 5, 1, 4, 1, 7, 1, 2,
       1, 3, 3, 1, 7, 1, 4, 1, 7, 4, 1, 1, 4, 2, 4, 4, 1, 1, 7, 1, 1, 5,
       2, 2, 3, 1, 2, 3, 2, 2, 1, 2, 1, 6, 2, 2, 4, 7, 2, 2, 2, 7, 1, 2,
       1, 7, 3, 0, 1, 3, 1, 2, 7, 1, 1, 1, 2, 1, 3, 2, 1, 2, 1, 3, 1, 6,
       2, 1, 3, 2, 1, 2, 1, 4, 1, 7, 2, 2, 1, 1, 5, 7, 2, 2, 1, 2, 7, 1,
       4, 1, 2, 4, 7, 2, 3, 2, 1, 3, 2, 1, 1, 1, 2,

In [108]:
# Define a custom accuracy evaluation function 
def evaluate(preds, y_test, variation):
    counter = 0
    for i in range(len(preds)):
        if y_test[i]-variation <= preds[i] <= y_test[i]+variation:
            counter += 1
        else: 
            counter += 0
    accuracy = counter / len(preds) 
    return accuracy
        
evaluate(list(preds), list(y_test), 1)

0.6498599439775911

In [109]:
import pickle
with open('salary_predict_model.pkl', 'wb') as file:
    pickle.dump(pipeline, file)